<a href="https://colab.research.google.com/github/MaliniVibin1502/SQL-projects/blob/main/Analysis_Chicago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is an analysis of Chicago area schools, socio-economic factors and the crime data. I have used SQL magic and pandas for the analysis of data within the dataset by loading the csv files to a database called Chicago.

In [1]:
!pip install sqlite3
import sqlite3
import pandas as pd

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [2]:
%load_ext sql


In [3]:
con = sqlite3.connect("Chicago.db")
cur = con.cursor()

In [4]:
%sql sqlite:///Chicago.db

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoCensusData.csv")
df.to_sql("CENSUS_DATA", con, if_exists='replace', index=False,method="multi")

df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoCrimeData.csv")
df.to_sql("CHICAGO_CRIME_DATA", con, if_exists='replace', index=False, method="multi")

df = pd.read_csv("https://raw.githubusercontent.com/MaliniVibin1502/SQL-projects/main/data/ChicagoPublicSchools.csv")
df.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists='replace', index=False, method="multi")

566

Find the total number of crimes recorded in the CRIME table.

In [6]:
%%sql
SELECT COUNT(DISTINCT(CASE_NUMBER)) AS TOTAL_CASES FROM CHICAGO_CRIME_DATA

 * sqlite:///Chicago.db
Done.


TOTAL_CASES
533


List community area names and numbers with per capita income less than 11000.

In [7]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME FROM CENSUS_DATA WHERE PER_CAPITA_INCOME < 11000

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


List all case numbers for crimes involving minors?

In [8]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_MINOR_CASES FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE '%MINOR'

 * sqlite:///Chicago.db
Done.


TOTAL_MINOR_CASES
2


List all kidnapping crimes involving a child?

In [9]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_CHILD_ADBUCTION FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE 'CHILD%'

 * sqlite:///Chicago.db
Done.


TOTAL_CHILD_ADBUCTION
1


List the kind of crimes that were recorded at schools.

In [10]:
%%sql
SELECT COUNT(CASE_NUMBER) AS TOTAL_CASES_REPORTED_AT_SCHOOL FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE "SCHOOL%"

 * sqlite:///Chicago.db
Done.


TOTAL_CASES_REPORTED_AT_SCHOOL
12


List the type of schools along with the average safety score for each type.

In [11]:
%%sql
SELECT "Elementary, Middle, or High School", ROUND(AVG(SAFETY_SCORE),2) AS AVERAGE_SCORE FROM CHICAGO_PUBLIC_SCHOOLS_DATA GROUP BY "Elementary, Middle, or High School" ORDER BY SAFETY_SCORE

 * sqlite:///Chicago.db
Done.


"Elementary, Middle, or High School",AVERAGE_SCORE
HS,49.62
MS,48.0
ES,49.52


List 5 community areas with highest % of households below poverty line

In [12]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY FROM CENSUS_DATA ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


Which community area is most crime prone?

In [13]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(CASE_NUMBER) AS TOTAL_CASES FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY TOTAL_CASES DESC LIMIT 1

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NUMBER,TOTAL_CASES
25.0,43


Use a sub-query to find the name of the community area with highest hardship index

In [14]:
%%sql
SELECT COMMUNITY_AREA_NAME, HARDSHIP_INDEX FROM CENSUS_DATA WHERE HARDSHIP_INDEX=(SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA)

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NAME,HARDSHIP_INDEX
Riverdale,98.0


 Use a sub-query to determine the Community Area Name with most number of crimes?

In [15]:
%%sql
SELECT COMMUNITY_AREA_NAME FROM CENSUS_DATA WHERE COMMUNITY_AREA_NUMBER = (SELECT COMMUNITY_AREA_NUMBER FROM CHICAGO_CRIME_DATA GROUP BY COMMUNITY_AREA_NUMBER ORDER BY COUNT(CASE_NUMBER) DESC LIMIT 1)

 * sqlite:///Chicago.db
Done.


COMMUNITY_AREA_NAME
Austin


List the school names, community names and average attendance for communities with a hardship index of 98.

In [16]:
%%sql
SELECT NAME_OF_SCHOOL, C.COMMUNITY_AREA_NAME, AVERAGE_STUDENT_ATTENDANCE
FROM CHICAGO_PUBLIC_SCHOOLS_DATA C
LEFT JOIN CENSUS_DATA CE ON C.COMMUNITY_AREA_NUMBER = CE.COMMUNITY_AREA_NUMBER
WHERE CE.HARDSHIP_INDEX = 98

 * sqlite:///Chicago.db
Done.


NAME_OF_SCHOOL,COMMUNITY_AREA_NAME,AVERAGE_STUDENT_ATTENDANCE
George Washington Carver Military Academy High School,RIVERDALE,91.60%
George Washington Carver Primary School,RIVERDALE,90.90%
Ira F Aldridge Elementary School,RIVERDALE,92.90%
William E B Dubois Elementary School,RIVERDALE,93.30%


 List all crimes that took place at a school. Include case number, crime type and community name.

In [17]:
%%sql
SELECT CASE_NUMBER,PRIMARY_TYPE,LOCATION_DESCRIPTION, S.COMMUNITY_AREA_NAME
FROM CHICAGO_CRIME_DATA C
LEFT JOIN CENSUS_DATA S ON  C.COMMUNITY_AREA_NUMBER = S.COMMUNITY_AREA_NUMBER
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///Chicago.db
Done.


CASE_NUMBER,PRIMARY_TYPE,LOCATION_DESCRIPTION,COMMUNITY_AREA_NAME
HL353697,BATTERY,"SCHOOL, PUBLIC, GROUNDS",South Shore
HL725506,BATTERY,"SCHOOL, PUBLIC, BUILDING",Lincoln Square
HP716225,BATTERY,"SCHOOL, PUBLIC, BUILDING",Douglas
HH639427,BATTERY,"SCHOOL, PUBLIC, BUILDING",Austin
JA460432,BATTERY,"SCHOOL, PUBLIC, GROUNDS",Ashburn
HS200939,CRIMINAL DAMAGE,"SCHOOL, PUBLIC, GROUNDS",Austin
HK577020,NARCOTICS,"SCHOOL, PUBLIC, GROUNDS",Rogers Park
HS305355,NARCOTICS,"SCHOOL, PUBLIC, BUILDING",Brighton Park
HT315369,ASSAULT,"SCHOOL, PUBLIC, GROUNDS",East Garfield Park
HR585012,CRIMINAL TRESPASS,"SCHOOL, PUBLIC, GROUNDS",Ashburn


Create a view showing the columns listed in the following table, with new column names as shown in the second column.

In [18]:
%%sql
CREATE VIEW SCHOOL_DATA (School_Name,Safety_Rating,Family_Rating,Environment_Rating,Instruction_Rating,Leaders_Rating,Teachers_Rating) AS
SELECT NAME_OF_SCHOOL,Safety_Icon,Family_Involvement_Icon,Environment_Icon,Instruction_Icon,Leaders_Icon,Teachers_Icon
FROM CHICAGO_PUBLIC_SCHOOLS_DATA;

SELECT School_Name, Leaders_Rating FROM SCHOOL_DATA

 * sqlite:///Chicago.db
Done.
Done.


School_Name,Leaders_Rating
Abraham Lincoln Elementary School,Weak
Adam Clayton Powell Paideia Community Academy Elementary School,Weak
Adlai E Stevenson Elementary School,Weak
Agustin Lara Elementary Academy,Weak
Air Force Academy High School,Weak
Albany Park Multicultural Academy,Weak
Albert G Lane Technical High School,Weak
Albert R Sabin Elementary Magnet School,Weak
Alcott High School for the Humanities,Weak
Alessandro Volta Elementary School,Weak
